In [ ]:
# import pandas as pd
# import os
# import time
# import shutil
# from pathlib import Path
# from typing import List, Tuple, Dict

# # Set ffmpeg/ffprobe paths BEFORE importing pydub
# os.environ["PATH"] = "/opt/homebrew/bin:/usr/local/bin:" + os.environ.get("PATH", "")

# class ChineseSyllableSynthesizerGTTS:
#     """
#     Chinese syllable synthesizer using Google Text-to-Speech (gTTS)
#     with audio processing to set sampling rate to 48000Hz and duration to ~220ms
#     """
#     def __init__(self, output_dir: str):
#         try:
#             from gtts import gTTS
#             self.gTTS = gTTS
#             self.available = True
#         except ImportError:
#             print("gTTS not installed. Install with: pip install gtts")
#             self.available = False

#         # Try to import scipy for audio processing
#         try:
#             from scipy.io import wavfile
#             import numpy as np
#             self.wavfile = wavfile
#             self.np = np
#             self.scipy_available = True
#         except ImportError:
#             print("scipy not installed. Install with: pip install scipy numpy")
#             self.scipy_available = False

#         # Try pydub as backup (requires ffmpeg)
#         try:
#             from pydub import AudioSegment
#             self.AudioSegment = AudioSegment
#             self.pydub_available = True
#             print("pydub is available")
#         except ImportError:
#             self.pydub_available = False
#             print("pydub not available")

#         self.output_dir = Path(output_dir)
#         self.output_dir.mkdir(parents=True, exist_ok=True)
#         self.target_sample_rate = 48000
#         self.target_duration_ms = 400

#     def process_audio_scipy(self, input_path: str, output_path: str) -> bool:
#         """
#         Process audio file using scipy (no ffmpeg required).
#         Returns True if successful, False otherwise.
#         """
#         if not self.scipy_available:
#             return False

#         try:
#             import subprocess
#             import tempfile
            
#             # Convert mp3 to wav using ffmpeg (if available) or use pydub
#             temp_wav = tempfile.NamedTemporaryFile(suffix='.wav', delete=False)
#             temp_wav_path = temp_wav.name
#             temp_wav.close()
            
#             # Try to convert mp3 to wav
#             if self.pydub_available:
#                 try:
#                     audio = self.AudioSegment.from_mp3(input_path)
#                     audio.export(temp_wav_path, format="wav")
#                 except Exception as e:
#                     print(f"pydub conversion failed: {e}")
#                     return False
#             else:
#                 return False
            
#             # Read wav file
#             sample_rate, data = self.wavfile.read(temp_wav_path)
            
#             # Convert to mono if stereo
#             if len(data.shape) > 1:
#                 data = data.mean(axis=1)
            
#             # Resample to 48000Hz if needed
#             if sample_rate != self.target_sample_rate:
#                 duration = len(data) / sample_rate
#                 num_samples = int(duration * self.target_sample_rate)
#                 data = self.np.interp(
#                     self.np.linspace(0, len(data), num_samples),
#                     self.np.arange(len(data)),
#                     data
#                 )
#                 sample_rate = self.target_sample_rate
            
#             # Adjust duration to 220ms
#             target_samples = int(self.target_sample_rate * self.target_duration_ms / 1000)
#             current_samples = len(data)
            
#             if current_samples > target_samples:
#                 # Trim to target duration
#                 data = data[:target_samples]
#             elif current_samples < target_samples:
#                 # Pad with zeros (silence)
#                 padding = self.np.zeros(target_samples - current_samples)
#                 data = self.np.concatenate([data, padding])
            
#             # Ensure data is in int16 format
#             data = data.astype(self.np.int16)
            
#             # Save as wav first
#             temp_output_wav = tempfile.NamedTemporaryFile(suffix='.wav', delete=False)
#             temp_output_wav_path = temp_output_wav.name
#             temp_output_wav.close()
            
#             self.wavfile.write(temp_output_wav_path, self.target_sample_rate, data)
            
#             # Convert wav back to mp3
#             if self.pydub_available:
#                 audio = self.AudioSegment.from_wav(temp_output_wav_path)
#                 audio.export(output_path, format="mp3", bitrate="128k")
#             else:
#                 # If no pydub, just copy the wav as mp3 (not ideal but works)
#                 shutil.copy2(temp_output_wav_path, output_path)
            
#             # Clean up temp files
#             try:
#                 os.unlink(temp_wav_path)
#                 os.unlink(temp_output_wav_path)
#             except:
#                 pass
            
#             return True
            
#         except Exception as e:
#             print(f"Error in scipy processing: {e}")
#             return False

#     def process_audio_simple(self, input_path: str, output_path: str) -> bool:
#         """
#         Simple processing without external dependencies - just copy the file.
#         This is a fallback that doesn't actually process the audio.
#         """
#         try:
#             shutil.copy2(input_path, output_path)
#             print(f"Warning: Audio not processed (missing dependencies), using original file")
#             return True
#         except Exception as e:
#             print(f"Error copying file: {e}")
#             return False

#     def synthesize_syllable(self, syllable: str, cache_dir: Path) -> str:
#         """Synthesize a single syllable using gTTS and save in cache_dir."""
#         if not self.available:
#             return None

#         import re
#         safe_syllable = re.sub(r'[^\w\-_.]', '_', syllable)
#         filename = f"{safe_syllable}.mp3"
#         filepath = cache_dir / filename

#         # Skip if already exists
#         if filepath.exists():
#             print(f"Using cached: {filepath}")
#             return str(filepath)

#         try:
#             # Generate with gTTS to a temporary file
#             temp_filepath = cache_dir / f"temp_{filename}"
#             tts = self.gTTS(text=syllable, lang='zh', slow=False, tld='com')
#             tts.save(str(temp_filepath))
            
#             # Try processing methods in order of preference
#             success = False
            
#             # Try scipy method first (no ffmpeg in PATH needed for conversion)
#             if self.scipy_available and self.pydub_available:
#                 success = self.process_audio_scipy(str(temp_filepath), str(filepath))
#                 if success:
#                     temp_filepath.unlink()
#                     print(f"Generated and processed (scipy): {filepath} (48000Hz, ~220ms)")
#                     return str(filepath)
            
#             # Fallback: just use the original file
#             if not success:
#                 shutil.move(str(temp_filepath), str(filepath))
#                 print(f"Generated (no processing - install scipy/numpy or ffmpeg): {filepath}")
            
#             return str(filepath)
            
#         except Exception as e:
#             print(f"Error generating {syllable}: {e}")
#             # Clean up temp file if it exists
#             if temp_filepath.exists():
#                 try:
#                     temp_filepath.unlink()
#                 except:
#                     pass
#             return None

# def process_trials_from_csv(
#     csv_path: str,
#     output_base: str = "/Users/yufang/WM_load/chinese_audio_output/three_4_syllable_words",
#     method: str = 'gtts'
# ):
#     """
#     Read a CSV where each row is a trial with columns:
#     W1,W2,W3,...,Cue_Word,Cue_Pos,Cue,Block

#     For each block, create a block folder. For each trial, create a trial folder under the block.
#     In each trial folder, create 'words' and 'cue' subfolders.
#     'words' stores every single character audio, 'cue' only stores cue audio.
    
#     Strategy: First generate all unique syllables to a cache folder, then copy them to trial folders.
#     """
#     # Read CSV
#     try:
#         df = pd.read_csv(csv_path, encoding='utf-8')
#     except Exception as e:
#         print(f"Error reading CSV: {e}")
#         return None

#     # Find all word columns (W1, W2, W3, ..., Wn)
#     word_cols = [col for col in df.columns if col.startswith('W') and col[1:].isdigit()]
#     # Required columns for cue and block
#     required_cols = ['Cue_Word', 'Cue', 'Cue_Pos', 'block']
#     for col in required_cols:
#         if col not in df.columns:
#             print(f"Column '{col}' not found. Available: {df.columns.tolist()}")
#             return None

#     # Initialize synthesizer
#     if method == 'gtts':
#         synthesizer = ChineseSyllableSynthesizerGTTS(output_base)
#         if not synthesizer.available:
#             return None
        
#         # Check what processing is available
#         print("\nAudio processing capabilities:")
#         print(f"  - scipy/numpy: {'Available' if synthesizer.scipy_available else 'Not available'}")
#         print(f"  - pydub: {'Available' if synthesizer.pydub_available else 'Not available'}")
#         if synthesizer.scipy_available and synthesizer.pydub_available:
#             print("  → Will process audio to 48000Hz, ~220ms")
#         else:
#             print("  → WARNING: Audio will not be processed (install scipy, numpy, and optionally pydub)")
#             print("  → To enable processing: pip install scipy numpy pydub")
#     else:
#         print(f"Only 'gtts' method is implemented in this script.")
#         return None

#     # Step 1: Collect all unique syllables from the CSV
#     print("\n=== Step 1: Collecting unique syllables ===")
#     unique_syllables = set()
#     for col in word_cols:
#         for word in df[col]:
#             word_str = str(word).strip()
#             if word_str and word_str != 'nan':
#                 for char in word_str:
#                     if char.strip():
#                         unique_syllables.add(char)
    
#     print(f"Found {len(unique_syllables)} unique syllables")
    
#     # Step 2: Generate all syllables to a cache directory
#     print("\n=== Step 2: Generating all syllable audio files ===")
#     cache_dir = Path(output_base) / "_syllable_cache"
#     cache_dir.mkdir(parents=True, exist_ok=True)
    
#     syllable_cache: Dict[str, str] = {}
#     for syllable in sorted(unique_syllables):
#         audio_path = synthesizer.synthesize_syllable(syllable, cache_dir)
#         if audio_path:
#             syllable_cache[syllable] = audio_path
#         time.sleep(0.5)  # Rate limiting
    
#     print(f"Generated {len(syllable_cache)} syllable audio files")

#     # Step 3: Process each trial and copy audio files from cache
#     print("\n=== Step 3: Organizing audio files into trial folders ===")
    
#     # Group by block
#     for block, block_df in df.groupby('block'):
#         block_dir = Path(output_base) / f"block_{block}"
#         block_dir.mkdir(parents=True, exist_ok=True)
#         print(f"\n=== Processing block {block} ===")

#         # trial index should start from 1 for each block
#         for trial_idx, (row_index, row) in enumerate(block_df.iterrows(), start=1):
#             trial_dir = block_dir / f"trial_{trial_idx}"
#             words_dir = trial_dir / "words"
#             cue_dir = trial_dir / "cue"
#             words_dir.mkdir(parents=True, exist_ok=True)
#             cue_dir.mkdir(parents=True, exist_ok=True)
#             print(f"\n--- Processing trial {trial_idx} in block {block} ---")

#             # Process all words (W1, W2, ..., Wn)
#             for word_idx, col in enumerate(word_cols):
#                 word = str(row[col]).strip()
#                 if not word or word == 'nan':
#                     print(f"Skipping empty word in {col}")
#                     continue
                
#                 # Copy each syllable from cache
#                 syllables = list(word)
#                 for syllable_idx, syllable in enumerate(syllables):
#                     if syllable.strip() and syllable in syllable_cache:
#                         import re
#                         safe_word = re.sub(r'[^\w\-_.]', '_', word)
#                         filename = f"word{word_idx+1}_syllable_{syllable_idx+1}_{safe_word}_{syllable}.mp3"
#                         src_path = Path(syllable_cache[syllable])
#                         dst_path = words_dir / filename
#                         shutil.copy2(src_path, dst_path)
#                         print(f"Copied: {syllable} -> {dst_path}")

#             # Process cue: use Cue_Word and Cue_Pos as integer indices
#             cue_word_idx = None
#             cue_char = None

#             # Get cue_word_idx and cue_pos as integers
#             try:
#                 cue_word_idx = int(row['Cue_Word']) - 1 if not pd.isnull(row['Cue_Word']) else None
#                 cue_pos = int(row['Cue_Pos']) if not pd.isnull(row['Cue_Pos']) else None
#             except Exception as e:
#                 print(f"Error parsing Cue_Word or Cue_Pos for trial {trial_idx}: {e}")
#                 cue_word_idx = None
#                 cue_pos = None

#             # Get the cue word and cue character
#             cue_word = None
#             if cue_word_idx is not None and 0 <= cue_word_idx < len(word_cols):
#                 cue_word_col = word_cols[cue_word_idx]
#                 cue_word = str(row[cue_word_col]).strip()
#                 if cue_word and cue_pos is not None and 0 <= cue_pos < len(cue_word):
#                     cue_char = cue_word[cue_pos]
#                 else:
#                     cue_char = None

#             if cue_word and cue_char and cue_word_idx is not None and cue_pos is not None:
#                 # The audio file in words_dir is named as:
#                 # word{word_idx+1}_syllable_{syllable_index+1}_{safe_word}_{syllable}.mp3
#                 import re
#                 safe_word = re.sub(r'[^\w\-_.]', '_', cue_word)
#                 filename = f"word{cue_word_idx+1}_syllable_{cue_pos+1}_{safe_word}_{cue_char}.mp3"
#                 src_path = words_dir / filename
#                 dst_path = cue_dir / filename
#                 if src_path.exists():
#                     shutil.copy2(src_path, dst_path)
#                     print(f"Copied cue audio from {src_path} to {dst_path}")
#                 else:
#                     print(f"Warning: Cue audio file {src_path} does not exist for trial {trial_idx}")
#             else:
#                 print(f"Warning: Cue word or cue character missing or not found for trial {trial_idx}")

#     print("\nAll trials processed. Audio files are in:", output_base)
#     print(f"Syllable cache is in: {cache_dir}")

# if __name__ == "__main__":
#     # Example usage - choose one of the three CSV files:
    
#     # Option 1: three_4_syllable_exp_with_blocks.csv
#     # csv_path = "Exp1_Syllable_Sequence_and_Cueing_Syllable_List/three_4_syllable_exp_with_blocks.csv"
#     # output_base = "/Users/yufang/WM_load/chinese_audio_output/three_4_syllable_words"
    
#     # Option 2: three_3_syllable_exp_with_blocks.csv
#     # csv_path = "Exp1_Syllable_Sequence_and_Cueing_Syllable_List/three_3_syllable_exp_with_blocks.csv"
#     # output_base = "/Users/yufang/WM_load/chinese_audio_output/three_3_syllable_words"
    
#     # Option 3: four_3_syllable_exp_with_blocks.csv
#     csv_path = "Exp1_Syllable_Sequence_and_Cueing_Syllable_List/four_3_syllable_exp_with_blocks.csv"
#     output_base = "/Users/yufang/WM_load/chinese_audio_output/four_3_syllable_words"
    
#     process_trials_from_csv(csv_path, output_base)

pydub is available

Audio processing capabilities:
  - scipy/numpy: Available
  - pydub: Available
  → Will process audio to 48000Hz, ~220ms

=== Step 1: Collecting unique syllables ===
Found 358 unique syllables

=== Step 2: Generating all syllable audio files ===
Generated and processed (scipy): /Users/yufang/WM_load/chinese_audio_output/four_3_syllable_words/_syllable_cache/业.mp3 (48000Hz, ~220ms)
Generated and processed (scipy): /Users/yufang/WM_load/chinese_audio_output/four_3_syllable_words/_syllable_cache/中.mp3 (48000Hz, ~220ms)
Generated and processed (scipy): /Users/yufang/WM_load/chinese_audio_output/four_3_syllable_words/_syllable_cache/主.mp3 (48000Hz, ~220ms)
Generated and processed (scipy): /Users/yufang/WM_load/chinese_audio_output/four_3_syllable_words/_syllable_cache/乐.mp3 (48000Hz, ~220ms)
Generated and processed (scipy): /Users/yufang/WM_load/chinese_audio_output/four_3_syllable_words/_syllable_cache/乒.mp3 (48000Hz, ~220ms)
Generated and processed (scipy): /Users/yufa

KeyboardInterrupt: 

In [10]:
import pandas as pd
import os
import time
import shutil
from pathlib import Path
from typing import List, Tuple, Dict

# Set ffmpeg/ffprobe paths BEFORE importing pydub
os.environ["PATH"] = "/opt/homebrew/bin:/usr/local/bin:" + os.environ.get("PATH", "")

class ChineseSyllableSynthesizerGTTS:
    """
    Chinese syllable synthesizer using Google Text-to-Speech (gTTS)
    with audio processing to set sampling rate to 48000Hz and duration to ~220ms
    """
    def __init__(self, output_dir: str):
        try:
            from gtts import gTTS
            self.gTTS = gTTS
            self.available = True
        except ImportError:
            print("gTTS not installed. Install with: pip install gtts")
            self.available = False

        # Try to import scipy for audio processing
        try:
            from scipy.io import wavfile
            import numpy as np
            self.wavfile = wavfile
            self.np = np
            self.scipy_available = True
        except ImportError:
            print("scipy not installed. Install with: pip install scipy numpy")
            self.scipy_available = False

        # Try pydub as backup (requires ffmpeg)
        try:
            from pydub import AudioSegment
            self.AudioSegment = AudioSegment
            self.pydub_available = True
            print("pydub is available")
        except ImportError:
            self.pydub_available = False
            print("pydub not available")

        self.output_dir = Path(output_dir)
        self.output_dir.mkdir(parents=True, exist_ok=True)
        self.target_sample_rate = 48000
        self.target_duration_ms = 350

    def process_audio_scipy(self, input_path: str, output_path: str) -> bool:
        """
        Process audio file using scipy.
        Trims silence from beginning/end, then speeds up to fit ~220ms while preserving pitch.
        Returns True if successful, False otherwise.
        """
        if not self.scipy_available:
            return False

        try:
            import subprocess
            import tempfile
            
            # Convert mp3 to wav using pydub
            temp_wav = tempfile.NamedTemporaryFile(suffix='.wav', delete=False)
            temp_wav_path = temp_wav.name
            temp_wav.close()
            
            # Try to convert mp3 to wav
            if self.pydub_available:
                try:
                    audio = self.AudioSegment.from_mp3(input_path)
                    
                    # Trim silence from beginning and end
                    # silence_thresh: consider anything quieter than -50 dBFS as silence
                    # min_silence_len: minimum length of silence to detect (50ms)
                    from pydub.silence import detect_leading_silence
                    
                    def trim_silence(audio_segment, silence_threshold=-50.0, chunk_size=10):
                        """
                        Trim silence from beginning and end of audio
                        """
                        start_trim = detect_leading_silence(audio_segment, silence_threshold=silence_threshold, chunk_size=chunk_size)
                        end_trim = detect_leading_silence(audio_segment.reverse(), silence_threshold=silence_threshold, chunk_size=chunk_size)
                        
                        duration = len(audio_segment)
                        trimmed = audio_segment[start_trim:duration-end_trim]
                        return trimmed
                    
                    # Trim silence
                    audio = trim_silence(audio, silence_threshold=-50.0, chunk_size=10)
                    
                    # Get current duration after trimming
                    current_duration_ms = len(audio)
                    
                    # Speed up to target duration if longer (preserves pitch)
                    if current_duration_ms > self.target_duration_ms:
                        speed_factor = current_duration_ms / self.target_duration_ms
                        # Use pydub's speedup which changes sample rate (preserves pitch)
                        audio = audio.speedup(playback_speed=speed_factor)
                    elif current_duration_ms < self.target_duration_ms:
                        # Pad with silence if shorter
                        silence_duration = self.target_duration_ms - current_duration_ms
                        silence = self.AudioSegment.silent(duration=silence_duration)
                        audio = audio + silence
                    
                    # Export to temp wav
                    audio.export(temp_wav_path, format="wav")
                    
                except Exception as e:
                    print(f"pydub conversion failed: {e}")
                    return False
            else:
                return False
            
            # Read wav file
            sample_rate, data = self.wavfile.read(temp_wav_path)
            
            # Convert to mono if stereo
            if len(data.shape) > 1:
                data = data.mean(axis=1)
            
            # Resample to 48000Hz if needed
            if sample_rate != self.target_sample_rate:
                duration = len(data) / sample_rate
                num_samples = int(duration * self.target_sample_rate)
                data = self.np.interp(
                    self.np.linspace(0, len(data), num_samples),
                    self.np.arange(len(data)),
                    data
                )
                sample_rate = self.target_sample_rate
            
            # Ensure data is in int16 format
            data = data.astype(self.np.int16)
            
            # Save as wav first
            temp_output_wav = tempfile.NamedTemporaryFile(suffix='.wav', delete=False)
            temp_output_wav_path = temp_output_wav.name
            temp_output_wav.close()
            
            self.wavfile.write(temp_output_wav_path, self.target_sample_rate, data)
            
            # Convert wav back to mp3
            if self.pydub_available:
                audio = self.AudioSegment.from_wav(temp_output_wav_path)
                audio.export(output_path, format="mp3", bitrate="128k")
            else:
                # If no pydub, just copy the wav as mp3 (not ideal but works)
                shutil.copy2(temp_output_wav_path, output_path)
            
            # Clean up temp files
            try:
                os.unlink(temp_wav_path)
                os.unlink(temp_output_wav_path)
            except:
                pass
            
            return True
            
        except Exception as e:
            print(f"Error in scipy processing: {e}")
            return False

    def process_audio_simple(self, input_path: str, output_path: str) -> bool:
        """
        Simple processing without external dependencies - just copy the file.
        This is a fallback that doesn't actually process the audio.
        """
        try:
            shutil.copy2(input_path, output_path)
            print(f"Warning: Audio not processed (missing dependencies), using original file")
            return True
        except Exception as e:
            print(f"Error copying file: {e}")
            return False

    def synthesize_syllable(self, syllable: str, cache_dir: Path) -> str:
        """Synthesize a single syllable using gTTS and save in cache_dir."""
        if not self.available:
            return None

        import re
        safe_syllable = re.sub(r'[^\w\-_.]', '_', syllable)
        filename = f"{safe_syllable}.mp3"
        filepath = cache_dir / filename

        # Skip if already exists
        if filepath.exists():
            print(f"Using cached: {filepath}")
            return str(filepath)

        try:
            # Generate with gTTS to a temporary file
            temp_filepath = cache_dir / f"temp_{filename}"
            tts = self.gTTS(text=syllable, lang='zh', slow=False, tld='com')
            tts.save(str(temp_filepath))
            
            # Try processing methods in order of preference
            success = False
            
            # Try scipy method first (no ffmpeg in PATH needed for conversion)
            if self.scipy_available and self.pydub_available:
                success = self.process_audio_scipy(str(temp_filepath), str(filepath))
                if success:
                    temp_filepath.unlink()
                    print(f"Generated and processed (scipy): {filepath} (48000Hz, ~220ms)")
                    return str(filepath)
            
            # Fallback: just use the original file
            if not success:
                shutil.move(str(temp_filepath), str(filepath))
                print(f"Generated (no processing - install scipy/numpy or ffmpeg): {filepath}")
            
            return str(filepath)
            
        except Exception as e:
            print(f"Error generating {syllable}: {e}")
            # Clean up temp file if it exists
            if temp_filepath.exists():
                try:
                    temp_filepath.unlink()
                except:
                    pass
            return None

def process_trials_from_csv(
    csv_path: str,
    output_base: str = "/Users/yufang/WM_load/chinese_audio_output/three_4_syllable_words",
    method: str = 'gtts'
):
    """
    Read a CSV where each row is a trial with columns:
    W1,W2,W3,...,Cue_Word,Cue_Pos,Cue,Block

    For each block, create a block folder. For each trial, create a trial folder under the block.
    In each trial folder, create 'words' and 'cue' subfolders.
    'words' stores every single character audio, 'cue' only stores cue audio.
    
    Strategy: First generate all unique syllables to a cache folder, then copy them to trial folders.
    """
    # Read CSV
    try:
        df = pd.read_csv(csv_path, encoding='utf-8')
    except Exception as e:
        print(f"Error reading CSV: {e}")
        return None

    # Find all word columns (W1, W2, W3, ..., Wn)
    word_cols = [col for col in df.columns if col.startswith('W') and col[1:].isdigit()]
    # Required columns for cue and block
    required_cols = ['Cue_Word', 'Cue', 'Cue_Pos', 'block']
    for col in required_cols:
        if col not in df.columns:
            print(f"Column '{col}' not found. Available: {df.columns.tolist()}")
            return None

    # Initialize synthesizer
    if method == 'gtts':
        synthesizer = ChineseSyllableSynthesizerGTTS(output_base)
        if not synthesizer.available:
            return None
        
        # Check what processing is available
        print("\nAudio processing capabilities:")
        print(f"  - scipy/numpy: {'Available' if synthesizer.scipy_available else 'Not available'}")
        print(f"  - pydub: {'Available' if synthesizer.pydub_available else 'Not available'}")
        if synthesizer.scipy_available and synthesizer.pydub_available:
            print("  → Will trim silence, then process audio to 48000Hz, ~220ms")
        else:
            print("  → WARNING: Audio will not be processed (install scipy, numpy, and optionally pydub)")
            print("  → To enable processing: pip install scipy numpy pydub")
    else:
        print(f"Only 'gtts' method is implemented in this script.")
        return None

    # Step 1: Collect all unique syllables from the CSV
    print("\n=== Step 1: Collecting unique syllables ===")
    unique_syllables = set()
    for col in word_cols:
        for word in df[col]:
            word_str = str(word).strip()
            if word_str and word_str != 'nan':
                for char in word_str:
                    if char.strip():
                        unique_syllables.add(char)
    
    print(f"Found {len(unique_syllables)} unique syllables")
    
    # Step 2: Generate all syllables to a cache directory
    print("\n=== Step 2: Generating all syllable audio files ===")
    cache_dir = Path(output_base) / "_syllable_cache"
    cache_dir.mkdir(parents=True, exist_ok=True)
    
    syllable_cache: Dict[str, str] = {}
    for syllable in sorted(unique_syllables):
        audio_path = synthesizer.synthesize_syllable(syllable, cache_dir)
        if audio_path:
            syllable_cache[syllable] = audio_path
        time.sleep(0.5)  # Rate limiting
    
    print(f"Generated {len(syllable_cache)} syllable audio files")

    # Step 3: Process each trial and copy audio files from cache
    print("\n=== Step 3: Organizing audio files into trial folders ===")
    
    # Group by block
    for block, block_df in df.groupby('block'):
        block_dir = Path(output_base) / f"block_{block}"
        block_dir.mkdir(parents=True, exist_ok=True)
        print(f"\n=== Processing block {block} ===")

        # trial index should start from 1 for each block
        for trial_idx, (row_index, row) in enumerate(block_df.iterrows(), start=1):
            trial_dir = block_dir / f"trial_{trial_idx}"
            words_dir = trial_dir / "words"
            cue_dir = trial_dir / "cue"
            words_dir.mkdir(parents=True, exist_ok=True)
            cue_dir.mkdir(parents=True, exist_ok=True)
            print(f"\n--- Processing trial {trial_idx} in block {block} ---")

            # Process all words (W1, W2, ..., Wn)
            for word_idx, col in enumerate(word_cols):
                word = str(row[col]).strip()
                if not word or word == 'nan':
                    print(f"Skipping empty word in {col}")
                    continue
                
                # Copy each syllable from cache
                syllables = list(word)
                for syllable_idx, syllable in enumerate(syllables):
                    if syllable.strip() and syllable in syllable_cache:
                        import re
                        safe_word = re.sub(r'[^\w\-_.]', '_', word)
                        filename = f"word{word_idx+1}_syllable_{syllable_idx+1}_{safe_word}_{syllable}.mp3"
                        src_path = Path(syllable_cache[syllable])
                        dst_path = words_dir / filename
                        shutil.copy2(src_path, dst_path)
                        print(f"Copied: {syllable} -> {dst_path}")

            # Process cue: use Cue_Word and Cue_Pos as integer indices
            cue_word_idx = None
            cue_char = None

            # Get cue_word_idx and cue_pos as integers
            try:
                cue_word_idx = int(row['Cue_Word']) - 1 if not pd.isnull(row['Cue_Word']) else None
                cue_pos = int(row['Cue_Pos']) if not pd.isnull(row['Cue_Pos']) else None
            except Exception as e:
                print(f"Error parsing Cue_Word or Cue_Pos for trial {trial_idx}: {e}")
                cue_word_idx = None
                cue_pos = None

            # Get the cue word and cue character
            cue_word = None
            if cue_word_idx is not None and 0 <= cue_word_idx < len(word_cols):
                cue_word_col = word_cols[cue_word_idx]
                cue_word = str(row[cue_word_col]).strip()
                if cue_word and cue_pos is not None and 0 <= cue_pos < len(cue_word):
                    cue_char = cue_word[cue_pos]
                else:
                    cue_char = None

            if cue_word and cue_char and cue_word_idx is not None and cue_pos is not None:
                # The audio file in words_dir is named as:
                # word{word_idx+1}_syllable_{syllable_index+1}_{safe_word}_{syllable}.mp3
                import re
                safe_word = re.sub(r'[^\w\-_.]', '_', cue_word)
                filename = f"word{cue_word_idx+1}_syllable_{cue_pos+1}_{safe_word}_{cue_char}.mp3"
                src_path = words_dir / filename
                dst_path = cue_dir / filename
                if src_path.exists():
                    shutil.copy2(src_path, dst_path)
                    print(f"Copied cue audio from {src_path} to {dst_path}")
                else:
                    print(f"Warning: Cue audio file {src_path} does not exist for trial {trial_idx}")
            else:
                print(f"Warning: Cue word or cue character missing or not found for trial {trial_idx}")

    print("\nAll trials processed. Audio files are in:", output_base)
    print(f"Syllable cache is in: {cache_dir}")

if __name__ == "__main__":
    # Example usage - choose one of the three CSV files:
    
    # Option 1: three_4_syllable_exp_with_blocks.csv
    csv_path = "Exp1_Syllable_Sequence_and_Cueing_Syllable_List/three_4_syllable_exp_with_blocks.csv"
    output_base = "/Users/yufang/WM_load/chinese_audio_output/three_4_syllable_words"
    
    # Option 2: three_3_syllable_exp_with_blocks.csv
    # csv_path = "Exp1_Syllable_Sequence_and_Cueing_Syllable_List/three_3_syllable_exp_with_blocks.csv"
    # output_base = "/Users/yufang/WM_load/chinese_audio_output/three_3_syllable_words"
    
    # Option 3: four_3_syllable_exp_with_blocks.csv
    # csv_path = "Exp1_Syllable_Sequence_and_Cueing_Syllable_List/four_3_syllable_exp_with_blocks.csv"
    # output_base = "/Users/yufang/WM_load/chinese_audio_output/four_3_syllable_words"
    
    process_trials_from_csv(csv_path, output_base)

pydub is available

Audio processing capabilities:
  - scipy/numpy: Available
  - pydub: Available
  → Will trim silence, then process audio to 48000Hz, ~220ms

=== Step 1: Collecting unique syllables ===
Found 375 unique syllables

=== Step 2: Generating all syllable audio files ===
Generated and processed (scipy): /Users/yufang/WM_load/chinese_audio_output/three_4_syllable_words/_syllable_cache/一.mp3 (48000Hz, ~220ms)
Generated and processed (scipy): /Users/yufang/WM_load/chinese_audio_output/three_4_syllable_words/_syllable_cache/七.mp3 (48000Hz, ~220ms)
Generated and processed (scipy): /Users/yufang/WM_load/chinese_audio_output/three_4_syllable_words/_syllable_cache/万.mp3 (48000Hz, ~220ms)
Generated and processed (scipy): /Users/yufang/WM_load/chinese_audio_output/three_4_syllable_words/_syllable_cache/三.mp3 (48000Hz, ~220ms)
Generated and processed (scipy): /Users/yufang/WM_load/chinese_audio_output/three_4_syllable_words/_syllable_cache/上.mp3 (48000Hz, ~220ms)
Generated and proces